In [11]:
import pandas as pd
import sqlite3

In [9]:
orders=pd.read_csv("orders.csv")
orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [10]:
user=pd.read_json("users.json")
user

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [12]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [14]:
restaurants

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [17]:
final=(orders.merge(user,on="user_id" ,how="left").merge(restaurants,on="restaurant_id",how="left"))
final

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [19]:
final.to_csv("final_food_delivery_dataset.csv",index=False)

In [94]:
final[final["membership"]=="Gold"].groupby("city")["total_amount"].sum()


city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [22]:
final.groupby("cuisine")["total_amount"].mean().idxmax()

'Mexican'

In [93]:
final.groupby("user_id")["total_amount"].sum().gt(1000).sum()


np.int64(2544)

In [34]:
final.groupby("rating")["total_amount"].sum().idxmax()


np.float64(4.8)

In [37]:
final.groupby("cuisine").agg(
    restaurant_count=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
).sort_values(["restaurant_count","revenue"])


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [46]:
round(final["membership"].eq("Gold").mean()*100)


50

In [59]:
final.groupby("restaurant_name_x").agg(
    avg_order=("total_amount","mean"),
    orders=("order_id","count")
).query("orders < 20").sort_values("avg_order", ascending=False).head(50)


,avg_order,orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
Hotel Dhaba Chinese,973.125556,18
Amma Mess Pure Veg,965.299444,18
Hotel Biryani Pure Veg,964.577692,13
Annapurna Curry House Multicuisine,954.512353,17


In [54]:
final.groupby(
    ["membership","cuisine"]
)["total_amount"].sum()


membership  cuisine
Gold        Chinese     977713.74
            Indian      979312.31
            Italian    1005779.05
            Mexican    1012559.79
Regular     Chinese     952790.91
            Indian      992100.27
            Italian    1018424.75
            Mexican    1072943.30
Name: total_amount, dtype: float64

In [53]:
final["order_date"] = pd.to_datetime(final["order_date"])
final["quarter"] = final["order_date"].dt.to_period("Q")

final.groupby("quarter")["total_amount"].sum().idxmax()


C:\Users\diksh\AppData\Local\Temp\ipykernel_18228\216157634.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final["order_date"] = pd.to_datetime(final["order_date"])


Period('2023Q3', 'Q-DEC')

In [90]:
final.groupby(final["membership"]=="Gold")["order_id"].count()

membership
False    5013
True     4987
Name: order_id, dtype: int64

In [65]:
round(final.groupby(final["city"]=="Hyderabad")["total_amount"].sum())

city
False    6122258.0
True     1889367.0
Name: total_amount, dtype: float64

In [67]:
final["user_id"].nunique()

2883

In [69]:
round(final.groupby(final["membership"]=="Gold")["order_id"].mean(),2)

membership
False    4998.77
True     5002.24
Name: order_id, dtype: float64

In [81]:
final[final["rating"] >=4.5]["order_id"].count()


np.int64(3374)

In [97]:
top_gold_city = final[final["membership"]=="Gold"].groupby("city")["total_amount"].sum().idxmax()


top_gold_city_orders = final[
    (final["membership"]=="Gold") & (final["city"]==top_gold_city)
]["order_id"].count()
top_gold_city_orders

np.int64(1337)